### 목표: RAG Q&A 동작 구축 → 그 다음에 LangGraph 연결 → 마지막에 Memory/ToolNode 확장

#### 1단계: RAG Q&A 동작 구축
- 벡터DB 생성 (Chroma/Pinecone)
- 임베딩 모델 선택
- retriever로 검색
- LLM에게 context + 질문 → 최종 답변 도출

[추가]
- 프롬프트 개선
- 게시글 데이터 처리


---

- DB 적재

In [ ]:
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv 
import json
import os 

load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')



# Embedding & Pinecone 연결
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "plant-qna" # _ 안됨
if index_name not in [idx["name"] for idx in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
db = pc.Index(index_name)



# 게시글 데이터 로드 & Document 변환
DATA_PATH = "../../cache/post_preproced_data.json"
with open(DATA_PATH, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

documents = []
for item in raw_data:
    text = f"Q: {item['question']}\nA: {item['answer']}"
    documents.append(
        Document(page_content=text, metadata=item.get("metadata", {}))
    )

In [25]:
len(documents)

838

In [ ]:
# Pinecone INSERT
for idx, docs in enumerate(documents):
    id_ = "groro" + "_" + str(idx + 1) + "_" + str(docs.metadata['post_id'])
    vector = embeddings.embed_query(docs.page_content)
    db.upsert([(
        id_, 
        vector, 
        {'text': docs.page_content, **docs.metadata}
    )])

# 838건, 12분

- 응답 생성(LCEL)

[pinecone](https://velog.io/@hoony19980/Langchain-Vectorstore-3-Pinecone) <br>
[langchain_pinecone](https://api.python.langchain.com/en/latest/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html)

In [58]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv 
import json
import os 

load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "plant-qna" 
index = pc.Index(index_name) 


# 리트리버
vector_store = PineconeVectorStore(index=index, embedding=OpenAIEmbeddings(model="text-embedding-3-small"))
retriever = vector_store.as_retriever(search_kwargs={"k": 3})


# 프롬프트
QNA_PROMPT = f"""
너는 식물에 대해 차분하게 상담해 주는 전문가이다.
아래 형식을 반드시 지키되, 실제 상담사가 말하듯 자연스럽고 단정적인 말투로 작성한다.
context 정보를 참고하여 상담을 진행한다.

### 답변 방식 ###
- 첫 문장은 사용자의 고민에 대한 핵심 답변을 한 줄로 요약한다. (채팅 응답처럼)
- 이후 이어지는 RAG 정보는 비슷한 사례의 해결 방향을 '요약 3줄'로 정리한다.
- 모든 문장은 따뜻하지만 과하지 않게, 실제 상담사가 말하듯 단정적으로 말한다.
- 마지막 문장은 대화를 이어가기 위해 질문형으로 마무리한다.

### 출력 형식 (반드시 유지) ###

[사용자의 상황을 판단해서 가장 핵심적인 조언을 한 문장으로 제시]
[현재 상황에 맞는 다음 추가 질문 유도]


[참고한 RAG 검색 정보]
{{context}}

사용자 질문: {{question}}
"""



prompt = ChatPromptTemplate.from_template(QNA_PROMPT)


# 모델
llm3 = ChatOpenAI(model="gpt-4o-mini", temperature=0.3) 


# 문서 포맷
def format_docs(docs):
    return "\n\n".join([f"문서{i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])


# 체인
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm3
    | StrOutputParser()
)

In [63]:
# 실행 테스트
def rag_query(question: str):
    return qa_chain.invoke(question)

queries = [
    "몬스테라 잎 끝이 갈색으로 타는데 물을 너무 많이 준 걸까?",
    "베란다 창문 쪽이 오전만 햇빛이 드는데 무슨 식물을 놓는 게 좋을까?",
    "산세베리아가 잘 안 자라는데 흙을 바꿔야 할까?",
    "화분 밑에 벌레 같은 게 생겨서 무섭다… 뿌리파리 같기도 한데 어떻게 없애?",
    "알로카시아 잎에 구멍이 나는데 병인지 자연스러운 건지 모르겠어.",
    "최근에 분갈이 했는데 잎이 다 처졌어. 회복 방법이 있을까?",
    "강아지가 화분을 자꾸 건드리는데 반려동물한테 안전한 식물 추천해줘.",
    "집이 너무 건조해서 가습 겸 키울 수 있는 식물이 있을까?",
    "흙 표면에 하얀 곰팡이 같은 게 생겨서 걱정돼… 이거 정상인가?",
    "키우기 쉬운 식물 중에 공기정화 잘 되는 애 추천해줄 수 있어?"
]

for q in queries:
    print("사용자:\n", q)
    print()
    response = rag_query(q)
    print("플래니:\n", response)
    print("-"*100)

사용자:
 몬스테라 잎 끝이 갈색으로 타는데 물을 너무 많이 준 걸까?

플래니:
 몬스테라 잎 끝이 갈색으로 타는 것은 과습이 원인일 수 있습니다. 물주기를 조절하고, 흙의 배수가 잘 되는지 확인해 보세요.

혹시 최근에 물주기를 얼마나 자주 하셨는지, 그리고 화분의 배수 상태는 어떤지 말씀해 주실 수 있나요?
----------------------------------------------------------------------------------------------------
사용자:
 베란다 창문 쪽이 오전만 햇빛이 드는데 무슨 식물을 놓는 게 좋을까?

플래니:
 오전 햇빛이 드는 베란다에는 햇빛을 좋아하는 식물이 적합합니다. 

오전 햇빛은 부드럽고 온화하여 많은 식물들이 잘 자랄 수 있는 조건입니다. 특히, 다육식물이나 허브류, 또는 일부 꽃 식물들이 이 환경에서 잘 자라곤 합니다. 식물의 특성에 따라 물주기와 관리 방법이 달라지므로, 어떤 식물을 원하시는지에 따라 선택이 달라질 수 있습니다. 

어떤 종류의 식물을 키우고 싶으신가요?
----------------------------------------------------------------------------------------------------
사용자:
 산세베리아가 잘 안 자라는데 흙을 바꿔야 할까?

플래니:
 산세베리아가 잘 자라지 않는다면 흙을 바꿔주는 것이 좋습니다. 

흙이 오래되거나 배수가 잘 되지 않는 경우, 뿌리가 건강하게 자라기 어려울 수 있습니다. 새 흙으로 교체할 때는 배수가 좋은 흙을 선택해 주시고, 흙의 물빠짐 상태를 잘 확인해 주세요. 또한, 과습을 피하기 위해 물주기를 조절하는 것도 중요합니다. 

혹시 현재 사용하고 있는 흙의 상태는 어떤가요?
----------------------------------------------------------------------------------------------------
사용자:
 화분 밑